# Datadog metrics analysis using datadog-api-client 
> install datadog-api-client dependency using command in the notebook itself !pip3 install datadog-api-client

> update __api_key, app_key, and api_host__

In [ ]:
from datetime import datetime
from datadog_api_client import ApiClient, Configuration
from datadog_api_client.v1.api.metrics_api import MetricsApi
import pandas as pd

configuration = Configuration()
configuration.api_key["apiKeyAuth"] = "6d219dd0bf16esimb229dd42a044772"
configuration.api_key["appKeyAuth"] = "94c66b397bcd6dsim37f80ad3b2fd44d4a08c4df"
configuration.server_variables["site"] = "us5.datadoghq.com"

> Specify the time period in epoch format over which you want to fetch the data. 

> time_delta in _sec_,  adjust this to get more granular data, but having a very low granularity can results in http/429 throttling limit error

Extraction Logic :

    - take the datadog query
    
    - pass the query to datadog api with a time span of time_delta milliseconds -> This would pull data in spans of T to T + time_delta
    
    - append this data to a pandas dataframe

In [ ]:
start = 1690113776  
end = 1690386584 

# run datadog metric collection for every 600 sec (10 min) window. 
# This will remove the granularity issue discussed in the blog. 
time_delta = 72000

> define query we want to run in datadog for data extraction

In [ ]:
cpu_sys_util_query = 'avg:system.cpu.system{*}' 
cpu_usr_util_query = 'avg:system.cpu.user{*}' 

In [ ]:
datetime_list = []
value_list = []
metric_list = []

def extract(query, start_at, end_at, time_delta):
        
    iteration_end = start_at + time_delta 

    while (iteration_end <= end_at):

        with ApiClient(configuration) as api_client:
            api_instance = MetricsApi(api_client)
            response = api_instance.query_metrics(
              #  _from=int((datetime.now() + relativedelta(second=-3)).timestamp()),
              #  to=int(datetime.now().timestamp()),
                _from=start_at,
                to=iteration_end,            
                query=query,
            )
        for datadog_result in response['series']:
            for time_value_pair_list in datadog_result['pointlist']:
                str_arr = str(time_value_pair_list)
                arr_x, arr_y = str_arr.split(',')
                converted_datetime = int(float(arr_x.replace('[','')))
                cntr_val = arr_y.replace(']','')
                datetime_list.append(converted_datetime/1000)
                value_list.append(float(cntr_val))
                metric_list.append(datadog_result['metric']) # store the query that was executed in datadog.

        start_at = iteration_end # change start time as end of last iteration
        iteration_end = iteration_end + time_delta # increment reading frame
            
        all_data = {
        'datetime': datetime_list,
        'value': value_list,
        'metric': metric_list
        }
    return pd.DataFrame.from_dict(all_data)

data_cpu_metrics = extract(cpu_sys_util_query, start, end, time_delta)
data_cpu_metrics2 = extract(cpu_usr_util_query, start, end, time_delta)

## Metrics visualization using seaborn.object

In [ ]:
import seaborn.objects as so

data_cpu_metrics['datetime'] = pd.to_datetime(data_cpu_metrics['datetime'], unit='s')
data_cpu_metrics2['datetime'] = pd.to_datetime(data_cpu_metrics2['datetime'], unit='s')

so.Plot(data_cpu_metrics, x="datetime", y="value",color="metric").add(so.Dots())
so.Plot(data_cpu_metrics2, x="datetime", y="value",color="metric").add(so.Dots())


_References:_

_https://shopify.engineering/export-datadog-metrics-in-jupyter-notebooks_

_https://github.com/DataDog/datadog-api-client-python/tree/master/examples/v1/usage-metering_

_https://github.com/DataDog/datadog-api-client-python_